In [1]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import re

In [ ]:
#Data preprocessing including meltPoint and addition of Sheet amino acid percentages and fix of helix amino acid percentages because S4pred counts from 1, python from 0
while os.path.basename(os.getcwd()) != 'topic04_02':
    os.chdir('..')
    print(os.getcwd())
path = './data'
prokaryotes = pd.read_csv(os.path.join(path, "prokaryotes_323columns.csv"))
droplist = [0,1,2,3,4,5,6,7,8,10,11,12,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,33,38,43,-1,-2,-3,-4,-5,-6,-12,-13,-14,-15,-16,-17,-18,-19,20,-21,-22,-23,-24]
col = prokaryotes.columns
Sheetind = pd.Series(np.zeros(prokaryotes.shape[0]))
Sheetind = prokaryotes['Sheet2'].apply(lambda x: list(map(int, re.findall(r'\d+', x) )) if type(x) == str else x)
Sheetseq = pd.Series(np.zeros(prokaryotes.shape[0]))
for n in range(len(Sheetseq)):
    Sheetseq[n] = np.array(list(prokaryotes.loc[n,'Sequence']))[Sheetind[n]]
prokaryotes = prokaryotes.drop(columns = ['helixind','helixseq'])
aacid = ['A', 'V', 'L', 'I', 'P', 'F', 'W', 'M', 'G', 'S', 'C', 'T', 'Y', 'N', 'Q', 'D', 'E', 'K', 'R', 'H']
for a in aacid:
    prokaryotes.drop(columns = f'{a}helix', inplace = True)
helixind = pd.Series(np.zeros(prokaryotes.shape[0]))
helixind = prokaryotes['Helix2'].apply(lambda x: np.array(list(map(int, re.findall(r'\d+', x) ))) -1 if type(x) == str else x)
for n in range(len(helixind)):
    if len(helixind[n]) != 0:
        prokaryotes.at[n,'helixseq'] = np.array(list(prokaryotes.loc[n,'Sequence']))[helixind[n]]
for a in aacid:
    prokaryotes[f'{a}helix'] = prokaryotes['helixseq'].apply(lambda x: list(x).count(a)/len(x) if type(x) == np.ndarray and np.ndim(x)!=0 else np.nan)
Sheetind = pd.Series(np.zeros(prokaryotes.shape[0]))
Sheetind = prokaryotes['Sheet2'].apply(lambda x: np.array(list(map(int, re.findall(r'\d+', x) ))) -1 if type(x) == str else x)
for n in range(len(Sheetseq)):
    if len(Sheetind[n])!=0:
        prokaryotes.at[n,'sheetseq'] = np.array(list(prokaryotes.loc[n,'Sequence']))[Sheetind[n]]
for a in aacid:
    prokaryotes[f'{a}sheet'] = prokaryotes['sheetseq'].apply(lambda x: list(x).count(a)/len(x) if type(x) == np.ndarray and np.ndim(x)!=0  and len(x) != 0 else np.nan)
if 'Unnamed: 0' in prokaryotes.columns:
    prokaryotes.drop(columns = ['Unnamed: 0.1','Unnamed: 0'], inplace = True)
prokaryotes.to_csv(os.path.join(path, "prokaryotes_348columns.csv"), index = False)